In [ ]:
import sys
sys.path.insert(0, "../src/")

from accelerate import Accelerator
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.pyplot as plt
import numpy as np
import torch

from models.unet import Unet
from diffuser.diffuser import Diffuser

# Generate

In [ ]:
accelerate = Accelerator(
    project_dir="../experiments",
)
model = Unet(in_channels=3, context_dim=5, channels=(64, 128))
model = accelerate.prepare(model)

In [ ]:
accelerate.load_state("../experiments/epoch_29/")
model.eval();

In [ ]:
diff = Diffuser(500, "cuda")

In [ ]:
# human, non-human, food, spell, side-facing
labels = [1.0, 0.0, 0.0, 0.0, 0.0]
cont = torch.tensor([labels], device=accelerate.device)
cont

In [ ]:
out = diff.sample_ddpm(model, 8, (16, 16), cont)

In [ ]:
def unorm(x):
    # unity norm. results in range of [0,1]
    # assume x (h,w,3)
    xmax = x.max((0,1))
    xmin = x.min((0,1))
    return(x - xmin)/(xmax - xmin)

def norm_all(store, n_t, n_s):
    # runs unity norm on all timesteps of all samples
    nstore = np.zeros_like(store)
    for t in range(n_t):
        for s in range(n_s):
            nstore[t,s] = unorm(store[t,s])
    return nstore

def plot_sample(x_gen_store,n_sample,nrows,save_dir, fn, w, save=False):
    ncols = n_sample//nrows
    sx_gen_store = np.moveaxis(x_gen_store,2,4)                               # change to Numpy image format (h,w,channels) vs (channels,h,w)
    nsx_gen_store = norm_all(sx_gen_store, sx_gen_store.shape[0], n_sample)   # unity norm to put in range [0,1] for np.imshow
    
    # create gif of images evolving over time, based on x_gen_store
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True,figsize=(ncols,nrows))
    def animate_diff(i, store):
        print(f'gif animating frame {i} of {store.shape[0]}', end='\r')
        plots = []
        for row in range(nrows):
            for col in range(ncols):
                axs[row, col].clear()
                axs[row, col].set_xticks([])
                axs[row, col].set_yticks([])
                plots.append(axs[row, col].imshow(store[i,(row*ncols)+col]))
        return plots
    ani = FuncAnimation(fig, animate_diff, fargs=[nsx_gen_store],  interval=200, blit=False, repeat=True, frames=nsx_gen_store.shape[0]) 
    plt.close()
    if save:
        ani.save(save_dir + f"{fn}_w{w}.gif", dpi=100, writer=PillowWriter(fps=5))
        print('saved gif at ' + save_dir + f"{fn}_w{w}.gif")
    return ani

In [ ]:
plt.clf()
animation_ddpm = plot_sample(out[::2, ...], 8, 2, "../experiments/", "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())